In [0]:
# !pip freeze
# would check the name & version of all libraries available for import

In [2]:
import pandas as pd
# from geopy.geocoders import Nominatim
from geopy import distance
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Metis Spring 2020/Hospitals.csv')
df_hosp = df.loc[(df.COUNTY == 'NEW YORK')]
df_hosp.sample(6)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
2399,-73.977079,40.736881,3400,10710010,VA MEDICAL CENTER - MANHATTAN CAMPUS,423 E 23RD ST,NEW YORK,NY,10010,NOT AVAILABLE,NOT AVAILABLE,MILITARY,OPEN,-999,NEW YORK,36061,USA,40.736881,-73.977079,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2015-05-22T00:00:00.000Z,http://www.nyharbor.va.gov/,NOT AVAILABLE,NOT AVAILABLE,36,GOVERNMENT - FEDERAL,-999,-999,NOT AVAILABLE,N
6013,-73.941624,40.803285,433,116810035,HENRY J. CARTER SPECIALTY HOSPITAL,1752 PARK AVENUE,NEW YORK,NY,10035,NOT AVAILABLE,(212) 318-8000,LONG TERM CARE,OPEN,201,NEW YORK,36061,USA,40.803285,-73.941624,622310,SPECIALTY (EXCEPT PSYCHIATRIC AND SUBSTANCE AB...,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2015-02-11T00:00:00.000Z,http://www.nychealthandhospitals.org/carter/ht...,7002050H,NOT AVAILABLE,36,NOT AVAILABLE,-999,201,NOT AVAILABLE,N
2392,-73.981771,40.733105,3393,121210003,MOUNT SINAI BETH ISRAEL,FIRST AVE AT 16TH STREET,NEW YORK,NY,10003,NOT AVAILABLE,(212) 420-2873,GENERAL ACUTE CARE,OPEN,825,NEW YORK,36061,USA,40.733105,-73.981771,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2015-02-24T00:00:00.000Z,http://www.bethisraelny.org/,7002002H,NOT AVAILABLE,36,NON-PROFIT,-999,825,NOT AVAILABLE,Y
2388,-73.973929,40.742047,3389,119410016,NYU HOSPITALS CENTER,550 FIRST AVENUE,NEW YORK,NY,10016,NOT AVAILABLE,(212) 263-5500,GENERAL ACUTE CARE,OPEN,844,NEW YORK,36061,USA,40.742047,-73.973929,622110,GENERAL MEDICAL AND SURGICAL HOSPITALS,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2015-02-24T00:00:00.000Z,http://nyulangone.org/locations/tisch-hospital,7002053H,NOT AVAILABLE,36,NOT AVAILABLE,-999,844,NOT AVAILABLE,N
3255,-73.942144,40.770302,6256,19010044,COLER-GOLDWATER SPECIALTY HOSPITAL & NURSING F...,900 MAIN STREET ROOSEVELT ISLAND,NEW YORK,NY,10044,NOT AVAILABLE,(212) 848-6000,LONG TERM CARE,CLOSED,210,NEW YORK,36061,USA,40.770302,-73.942144,622310,"EXTENDED CARE HOSPITALS (EXCEPT MENTAL, SUBSTA...",https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2014-02-10T00:00:00.000Z,http://www.nychealthandhospitals.org/coler/,7002051H,NOT AVAILABLE,36,GOVERNMENT - LOCAL,-999,210,NOT AVAILABLE,N
5995,-73.986808,40.769412,415,18810019,THE ADDICTION INSTITUTE OF NEW YORK,1000 10TH AVE,NEW YORK,NY,10019,NOT AVAILABLE,(212) 256-3800,PSYCHIATRIC,CLOSED,-999,NEW YORK,36061,USA,40.769412,-73.986808,622210,"HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)",https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY,2014-02-10T00:00:00.000Z,http://www.stlukeshospitalnyc.org/Addiction.aspx,1466,NOT AVAILABLE,36,NON-PROFIT,-999,-999,NOT AVAILABLE,N


In [0]:
stat_df = pd.read_csv('/content/drive/My Drive/Metis Spring 2020/MTAstations.csv')

In [5]:
stat_df['the_geom'].tail()

468     POINT (-73.9812359981396 40.57728100006751)
469    POINT (-74.00219709442206 40.75544635961596)
470    POINT (-73.95836178682246 40.76880251014895)
471    POINT (-73.95177090964917 40.77786104333163)
472    POINT (-73.9470660219183 40.784236650177654)
Name: the_geom, dtype: object

In [0]:
stat_coords_df = pd.DataFrame(stat_df.the_geom.str.split(' ',2).tolist(), columns = ['junk','long','lat'])

In [7]:
stat_coords_df.tail()

,junk,long,lat
468,POINT,(-73.9812359981396,40.57728100006751)
469,POINT,(-74.00219709442206,40.75544635961596)
470,POINT,(-73.95836178682246,40.76880251014895)
471,POINT,(-73.95177090964917,40.77786104333163)
472,POINT,(-73.9470660219183,40.784236650177654)


In [8]:
stat_df['long'] = pd.Series(stat_coords_df.long.str.strip('(')).astype(float)
stat_df['lat'] = pd.Series(stat_coords_df.lat.str.strip(')')).astype(float)
stat_df.tail(3)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,long,lat
470,http://web.mta.info/nyct/service/,641,72nd St,POINT (-73.95836178682246 40.76880251014895),Q,Q-all times,-73.958362,40.768803
471,http://web.mta.info/nyct/service/,642,86th St,POINT (-73.95177090964917 40.77786104333163),Q,Q-all times,-73.951771,40.777861
472,http://web.mta.info/nyct/service/,643,96th St,POINT (-73.9470660219183 40.784236650177654),Q,Q-all times,-73.947066,40.784237


In [9]:
stat_df.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,long,lat
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-73.991070,40.730054
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-74.000193,40.718803
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights",-73.983849,40.761728
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times",-73.974999,40.680862
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times",-73.894886,40.664714


In [10]:
stat_loc = pd.Series(zip(stat_df.lat, stat_df.long))
stat_loc.head()

0    (40.73005400028978, -73.99106999861966)
1    (40.71880300107709, -74.00019299927328)
2    (40.76172799961419, -73.98384899986625)
3    (40.68086213682956, -73.97499915116808)
4    (40.66471445143568, -73.89488591154061)
dtype: object

In [11]:
stat_loc_df = pd.DataFrame(stat_df['NAME'])
stat_loc_df.head()

,NAME
0,Astor Pl
1,Canal St
2,50th St
3,Bergen St
4,Pennsylvania Ave


In [13]:
stat_loc_df['lines'] = stat_df['LINE']
stat_loc_df['coords'] = stat_loc
stat_loc_df.head()

,NAME,lines,coords
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)"
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)"
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)"
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)"
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)"


Right. Haven't been doing any kind of job of commenting thus far. I have so far pulled in both the hospital and the MTA station location data and have cleaned up the latter to form ordered pairs suitable for geopy. I need to do that for the hospital data as well, then get to work writing a routine for finding the shortest distance from a station to one of the hospitals on the list.

We can either pick out the additional hospitals from Kings and Queens counties and whatever county names the Bronx and Staten Island have, or we can confine the station data to just Manhattan Island by finding some code somewhere that identifies points within a given polygon. PAG

First, let's knock out the closed hospitals.

In [14]:
still_open = (df_hosp.STATUS == 'OPEN')
still_open[5995] # The Addiction Institute of New York, which I noticed was 
                 # closed in my sample

False

In [15]:
df_hosp_open = df_hosp[still_open]
print(df_hosp.shape)
df_hosp_open.shape

(29, 34)


(25, 34)

In [16]:
df_hosp_open.head(2)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,LATITUDE,LONGITUDE,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
1153,-73.953021,40.769717,4154,18910021,GRACIE SQUARE HOSPITAL,"420 E 76TH ST, 2ND, 3RD, 4TH & 5TH FLOORS",NEW YORK,NY,10021,NOT AVAILABLE,(212) 988-4400,PSYCHIATRIC,OPEN,157,NEW YORK,36061,USA,40.769717,-73.953021,622210,PSYCHIATRIC AND SUBSTANCE ABUSE HOSPITALS,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2014-02-10T00:00:00.000Z,http://www.nygsh.org,1458,NOT AVAILABLE,36,NON-PROFIT,-999,157,NOT AVAILABLE,N
2350,-73.952315,40.764923,3351,18010021,HOSPITAL FOR SPECIAL SURGERY,535 EAST 70TH ST,NEW YORK,NY,10021,NOT AVAILABLE,(212) 606-1236,SPECIAL,OPEN,215,NEW YORK,36061,USA,40.764923,-73.952315,622310,SPECIALTY (EXCEPT PSYCHIATRIC AND SUBSTANCE AB...,https://profiles.health.ny.gov/hospital/county...,2018-08-20T00:00:00.000Z,IMAGERY/OTHER,2014-02-10T00:00:00.000Z,http://www.hss.edu,7002012H,NOT AVAILABLE,36,NON-PROFIT,-999,215,NOT AVAILABLE,N


Now we'll zip up the coordinates for the hospital locations just like we did for station locations. The only problem is the indexing.

In [17]:
hosp_loc = pd.Series(zip(df_hosp_open.Y, df_hosp_open.X), index = df_hosp_open.index)
hosp_loc.head()

1153     (40.7697165, -73.95302135)
2350    (40.76492338, -73.95231485)
2351    (40.78492694, -73.94453023)
2352    (40.73185306, -73.98448331)
2353    (40.78976753, -73.95321427)
dtype: object

In [0]:
hosp_loc_df = pd.DataFrame(df_hosp_open['NAME'])
hosp_loc_df['address'] = df_hosp_open['ADDRESS']
hosp_loc_df['coords'] = hosp_loc

In [19]:
hosp_loc_df.head()

,NAME,address,coords
1153,GRACIE SQUARE HOSPITAL,"420 E 76TH ST, 2ND, 3RD, 4TH & 5TH FLOORS","(40.7697165, -73.95302135)"
2350,HOSPITAL FOR SPECIAL SURGERY,535 EAST 70TH ST,"(40.76492338, -73.95231485)"
2351,METROPOLITAN HOSPITAL CENTER,1901 1ST AVE,"(40.78492694, -73.94453023)"
2352,NEW YORK EYE AND EAR INFIRMARY,310 E 14TH ST,"(40.73185306, -73.98448331)"
2353,MOUNT SINAI HOSPITAL,1 GUSTAVE L LEVY PL,"(40.78976753, -73.95321427)"


In [77]:
hosp_loc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1153 to 6396
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   NAME     25 non-null     object
 1   address  25 non-null     object
 2   coords   25 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


Now the task is basically to create columns in the station dataframe with the name and distance of the closest hospital to each station. Iterate over each station, find the distance to each hospital, save the name and distance of the closest hospital in the station dataframe.

In [21]:
# first let's see if we can get a single freaking distance

stat_loc_df['coords']

0      (40.73005400028978, -73.99106999861966)
1      (40.71880300107709, -74.00019299927328)
2      (40.76172799961419, -73.98384899986625)
3      (40.68086213682956, -73.97499915116808)
4      (40.66471445143568, -73.89488591154061)
                        ...                   
468     (40.57728100006751, -73.9812359981396)
469    (40.75544635961596, -74.00219709442206)
470    (40.76880251014895, -73.95836178682246)
471    (40.77786104333163, -73.95177090964917)
472    (40.784236650177654, -73.9470660219183)
Name: coords, Length: 473, dtype: object

In [22]:
stat_loc_df[0:20]

,NAME,lines,coords
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)"
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)"
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)"
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)"
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)"
5,238th St,1,"(40.88466700064975, -73.90087000018522)"
6,Cathedral Pkwy (110th St),A-B-C,"(40.800581558114956, -73.95806670661364)"
7,Kingston - Throop Aves,A-C,"(40.67991899941601, -73.94085899871263)"
8,65th St,E-M-R,"(40.74971952935675, -73.8987883783301)"
9,36th St,E-M-R,"(40.75196004401078, -73.92901818461539)"


In [23]:
stat_loc_df.iloc[0,2]

(40.73005400028978, -73.99106999861966)

In [24]:
distance.distance(stat_loc_df.iloc[0,2],hosp_loc_df.iloc[0,2]).miles

3.3877428119307282

In [0]:
# code to try the .apply(...) method to create a series of distances

def get_minimum_distance(location1, location_series, max_dist = 10):
  """Takes a coordinate tuple for location1, 
  a series of location names and coord tuples,
  and optionally a maximum distance.
  Outputs a list containing the distance to and index of the closest location
  in the series to location1."""
  dist = max_dist
  coords = None
  print(type(location_series))
  for location in location_series:
    mi_away = distance.distance(location1, location).miles
    if mi_away < dist:
      dist = mi_away
      coords = location
  return [dist, location_series[location_series == coords].index[0]]

In [26]:
stat_loc_df.shape

(473, 3)

In [36]:
test_list = get_minimum_distance(stat_loc_df.iloc[0,2], hosp_loc_df.iloc[:,2])
test_list[1]

<class 'pandas.core.series.Series'>


2352

In [73]:
# pull the name corresponding to that nearest hospital to station 0
type(hosp_loc_df['NAME'].loc[test_list[1]])

str

Somehow, I have to iterate over each location in the station location dataframe, run this function, push the returned distance into one new column and the name called back by index into a second new column. There should be a way to do this with .apply().

Let's rewrite that function to make it easier.

In [0]:
def get_mindist_name(location1, loc_series, name_series, max_dist = 10):
  """Takes a coordinate tuple for location1, 
  a series of coord tuples, an identically indexed series of names,
  and optionally a maximum distance.
  Outputs a list containing the distance to and index of the closest location
  in the series to location1."""
  dist = max_dist
  coords = None
  for location in loc_series:
    mi_away = distance.distance(location1, location).miles
    if mi_away < dist:
      dist = mi_away
      coords = location
  name = name_series[loc_series[loc_series == coords].index[0]]
  return [dist, name]

In [83]:
stat_hosp_df = pd.DataFrame(stat_loc_df['coords'].apply(
    get_mindist_name, loc_series = hosp_loc_df['coords'], name_series =
    hosp_loc_df['NAME']).tolist())

IndexError: ignored

In [76]:
stat_hosp_df.head(3)

,0,1
0,0.367354,2352 NEW YORK EYE AND EAR INFIRMARY Name: N...
1,0.631470,5165 NEW YORK-PRESBYTERIAN/LOWER MANHATTAN ...
2,0.588702,"5163 MOUNT SINAI WEST Name: NAME, dtype: ob..."


In [53]:
stat_hosp_full_df = pd.concat([stat_loc_df, stat_hosp_df], axis=1, sort=False)
stat_hosp_full_df.sample(10)

,NAME,lines,coords,0,1
411,163rd St - Amsterdam Av,A-C,"(40.83601299923096, -73.93989200188344)",0.268848,5994 NEW YORK PRESBYTERIAN MORGAN STANLEY C...
187,169th St,F,"(40.710517502784, -73.79347419927721)",8.984417,2914 MANHATTAN PSYCHIATRIC CENTER Name: NAM...
248,Elder Ave,6,"(40.82858400108929, -73.87915899874777)",3.331890,"6012 HARLEM HOSPITAL CENTER Name: NAME, dty..."
350,Lexington Ave - 63rd St,F-Q,"(40.76461809442373, -73.96608964413245)",0.498672,6396 MEMORIAL HOSPITAL FOR CANCER AND ALLIE...
289,3rd Ave - 149th St,2-5,"(40.816029252510006, -73.91779152760981)",1.163393,"6012 HARLEM HOSPITAL CENTER Name: NAME, dty..."
421,Rector St,R-W,"(40.707744756294474, -74.01297456253795)",0.446173,5165 NEW YORK-PRESBYTERIAN/LOWER MANHATTAN ...
39,Gun Hill Rd,2-5,"(40.877839385172024, -73.86613410538703)",2.477449,5155 NEW YORK PRESBYTERIAN HOSPITAL - ALLEN...
111,Church Ave,B-Q,"(40.65049324646484, -73.96288246192114)",4.703028,5165 NEW YORK-PRESBYTERIAN/LOWER MANHATTAN ...
215,Livonia Ave,L,"(40.66405727094644, -73.90056237226057)",6.367198,5165 NEW YORK-PRESBYTERIAN/LOWER MANHATTAN ...
293,40th St,7,"(40.74378100149132, -73.9240159984882)",2.081664,2350 HOSPITAL FOR SPECIAL SURGERY Name: NAM...


In [85]:
stat_hosp_full_df.columns = ['Station Name', 'Lines', 'Lat / Long', 'Hospital Distance (mi)', 'Hospital ID']
stat_hosp_full_df.tail(3)

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
470,72nd St,Q,"(40.76880251014895, -73.95836178682246)",0.287176,"1153 GRACIE SQUARE HOSPITAL Name: NAME, dty..."
471,86th St,Q,"(40.77786104333163, -73.95177090964917)",0.557846,"5143 LENOX HILL HOSPITAL Name: NAME, dtype:..."
472,96th St,Q,"(40.784236650177654, -73.9470660219183)",0.141272,2351 METROPOLITAN HOSPITAL CENTER Name: NAM...


In [0]:
stat_hosp_full_df.to_csv('/content/drive/My Drive/Metis Spring 2020/station_hospital_distances.csv')